# 数据聚合和分组操作

对数据集进行分类并将功能应用于每个组（无论是聚合还是转换），通常是数据分析工作流的关键组成部分。加载，合并和准备数据集后，您可能需要计算组统计信息或可能地透视表以用于报告或可视化目的。pandas提供了灵活的groupby界面，使您能够以自然的方式对数据集进行切片，切块和汇总。

关系数据库和SQL（代表“结构化查询语言”）流行的原因之一是数据的连接，过滤，转换和聚合的简便性。但是，诸如SQL之类的查询语言在某种程度上可以执行的组操作有所限制。如您所见，借助Python和pandas的表现力，我们可以利用任何接受pandas对象或NumPy数组的函数来执行相当复杂的组操作。在本章中，您将学习如何：

- 使用一个或多个键（以函数，数组或DataFrame列名的形式）将pandas对象拆分为多个部分
- 计算组摘要统计信息，例如计数，均值或标准差，或用户定义的函数
- 应用组内转换或其他操作，例如归一化，线性回归，等级或子集选择
- 计算数据透视表和交叉表
- 执行分位数分析和其他统计组分析

> 时间序列数据的聚合是groupby的一种特殊用例，在本书中称为重采样，第11章将对此进行单独处理。

## 10.1 GroupBy 机制

Hadley Wickham是R编程语言的许多流行软件包的作者，创造了术语split-apply-combine来描述组操作。在该过程的第一阶段，根据提供的一个或多个键，将熊猫对象中包含的数据（无论是系列，数据框还是其他）分为几组。分割是在对象的特定轴上执行的。例如，一个DataFrame可以按其行（axis = 0）或其列（axis = 1）分组。完成此操作后，会将一个函数应用于每个组，从而产生一个新值。最后，所有这些功能应用程序的结果都组合成一个结果对象。最终对象的形式通常取决于对数据执行的操作。有关简单组聚合的模型，请参见图10-1。

![10-1.bmp](images/10-1.bmp)

每个分组键可以采用多种形式，并且键不必都具有相同的类型：

- 与要分组的轴长度相同的值的列表或数组
- 一个值，指示DataFrame中的列名
- 一个dict或Series，在被分组的轴上的值和组名之间具有对应关系
- 在轴索引或索引中的各个标签上调用的函数

请注意，后三种方法是生成用于拆分对象的值数组的快捷方式。 如果这一切看起来都很抽象，请不要担心。 在本章中，我将给出所有这些方法的许多示例。 首先，这是一个小的表格数据集，作为DataFrame：

In [1]:
import numpy as np
import pandas as pd

df = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'],
                   'key2': ['one', 'two', 'one', 'two', 'one'],
                   'data1': np.random.randn(5),
                   'data2': np.random.randn(5)})

In [2]:
df

,key1,key2,data1,data2
0,a,one,-1.299674,0.592971
1,a,two,1.236301,-0.816862
2,b,one,0.247436,0.887067
3,b,two,-0.126520,1.445629
4,a,one,-0.894032,0.495924


假设您想使用key1中的标签来计算data1列的平均值。有很多方法可以做到这一点。一种是访问data1并通过key1的（系列）列调用groupby：

In [3]:
grouped = df['data1'].groupby(df['key1'])

grouped

现在，此 grouped 变量是一个GroupBy对象。除了关于组密钥 `df['key1']` 的一些中间数据外，它实际上还没有计算任何东西。想法是，该对象具有将某些操作应用于每个组所需的所有信息。例如，要计算组均值，我们可以调用GroupBy的mean方法：

In [4]:
grouped.mean()

key1
a   -0.319135
b    0.060458
Name: data1, dtype: float64

稍后，我将详细解释当您调用.mean（）时会发生什么。此处重要的是，已根据组键聚合了数据（系列），从而生成了一个新的系列，该系列现在由key1列中的唯一值索引。

结果索引的名称为'key1'，因为DataFrame列df ['key1']确实如此。

相反，如果我们将多个数组作为列表传递，则会得到不同的结果：

In [5]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()

In [6]:
means

key1  key2
a     one    -1.096853
      two     1.236301
b     one     0.247436
      two    -0.126520
Name: data1, dtype: float64

在这里，我们使用两个键对数据进行分组，并且得到的系列现在具有由观察到的唯一键对组成的层次结构索引：

In [7]:
means.unstack()

key2,one,two
key1,,
a,-1.096853,1.236301
b,0.247436,-0.126520


在此示例中，组键都是系列，尽管它们可以是长度合适的任何数组：

In [8]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])

df['data1'].groupby([states, years]).mean()

California  2005    1.236301
            2006    0.247436
Ohio        2005   -0.713097
            2006   -0.894032
Name: data1, dtype: float64

通常，分组信息与您要处理的数据位于同一DataFrame中。在这种情况下，您可以将列名（无论是字符串，数字还是其他Python对象）传递为组键：

In [9]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.319135,0.090678
b,0.060458,1.166348


In [10]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -1.096853  0.544448
     two   1.236301 -0.816862
b    one   0.247436  0.887067
     two  -0.126520  1.445629

在第一种情况下，您可能已经注意到df.groupby（'key1'）.mean（），结果中没有key2列。 因为df ['key2']不是数字数据，所以它被认为是令人讨厌的列，因此将其从结果中排除。 默认情况下，所有数字列都是聚合的，不过您可以很快看到它可以过滤到一个子集。

不管使用groupby的目的如何，通常有用的GroupBy方法是size，它返回一个包含组大小的Series：

In [11]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

请注意，组键中所有缺少的值都将从结果中排除。

### 遍历组

GroupBy对象支持迭代，生成一个2元组的序列，其中包含组名以及数据块。考虑以下：

In [12]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -1.299674  0.592971
1    a  two  1.236301 -0.816862
4    a  one -0.894032  0.495924
b
  key1 key2     data1     data2
2    b  one  0.247436  0.887067
3    b  two -0.126520  1.445629


如果有多个键，则元组中的第一个元素将是键值的元组：

In [13]:
for (key1, key2), group in df.groupby(['key1', 'key2']):
    print(key1, key2)
    print(group)

a one
  key1 key2     data1     data2
0    a  one -1.299674  0.592971
4    a  one -0.894032  0.495924
a two
  key1 key2     data1     data2
1    a  two  1.236301 -0.816862
b one
  key1 key2     data1     data2
2    b  one  0.247436  0.887067
b two
  key1 key2    data1     data2
3    b  two -0.12652  1.445629


当然，您可以选择对数据片段进行任何操作。 您可能会发现有用的食谱是将数据片段的字典作为单行计算：

In [14]:
pieces = dict(list(df.groupby('key1')))

In [15]:
pieces['b']

,key1,key2,data1,data2
2,b,one,0.247436,0.887067
3,b,two,-0.126520,1.445629


默认情况下，在axis = 0上对groupby groups进行分组，但是您可以在其他任何轴上进行分组。 例如，我们可以在此处按dtype将示例df的列分组，如下所示：

In [16]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [17]:
grouped = df.groupby(df.dtypes, axis=1)

我们可以这样打印组：

In [18]:
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0 -1.299674  0.592971
1  1.236301 -0.816862
2  0.247436  0.887067
3 -0.126520  1.445629
4 -0.894032  0.495924
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### 选择列或列子集

使用列名或列名数组对从DataFrame创建的GroupBy对象建立索引会对列子集的聚合产生影响。这意味着：

```python
    df.groupby('key1')['data1']
    df.groupby('key2')['data2']
```

是用于以下方面的语法糖：

```python
    df['data1'].groupby(df['key1'])
    df['data2'].groupby(df['key2'])
```

特别是对于大型数据集，可能希望仅汇总几列。例如，在前面的数据集中，要仅计算data2列的均值并将结果作为DataFrame获得，我们可以编写：

In [19]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.544448
     two  -0.816862
b    one   0.887067
     two   1.445629

如果传递了列表或数组，则此索引操作返回的对象是分组的DataFrame；如果仅将单个列名称作为标量传递，则该分组的数据对象是分组的Series：

In [20]:
s_grouped = df.groupby(['key1', 'key2'])['data2']

s_grouped

In [21]:
s_grouped.mean()

key1  key2
a     one     0.544448
      two    -0.816862
b     one     0.887067
      two     1.445629
Name: data2, dtype: float64

### 用字典和系列分组

分组信息可能以数组以外的形式存在。让我们考虑另一个示例DataFrame：

In [22]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values

In [23]:
people

,a,b,c,d,e
Joe,-0.631809,0.909270,-0.805212,0.204254,1.903409
Steve,0.525713,0.857140,0.981396,-0.438240,0.725576
Wes,0.333789,NaN,NaN,0.480433,-1.014250
Jim,0.008168,0.127904,-1.366128,1.347528,0.290832
Travis,1.747038,-2.069927,0.817761,0.380805,-0.198778


现在，假设我对各列有一个组对应关系，并希望按组将各列加起来：

In [24]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue', 
           'd': 'blue', 'e': 'red', 'f': 'orange'}

现在，您可以从此dict构造一个数组以传递给groupby，但是我们可以仅传递dict（我添加了键“ f”以突出显示未使用的分组键是可以的）：

In [25]:
by_column = people.groupby(mapping, axis=1)

In [26]:
by_column.sum()

,blue,red
Joe,-0.600959,2.180870
Steve,0.543155,2.108429
Wes,0.480433,-0.680461
Jim,-0.018600,0.426904
Travis,1.198566,-0.521667


系列具有相同的功能，可以将其视为固定大小的映射：

In [27]:
map_series = pd.Series(mapping)

map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [28]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### 按函数分组

与dict或Series相比，使用Python函数是定义组映射的更通用的方法。 每个索引值都会调用一次作为组键传递的任何函数，并将返回值用作组名。 更具体地说，请考虑上一节中的示例DataFrame，该示例以人们的名字作为索引值。 假设您想按名称的长度分组； 虽然您可以计算一个字符串长度的数组，但只需传递len函数会更简单：

In [29]:
people.groupby(len).sum()

,a,b,c,d,e
3,-0.289852,1.037173,-2.171340,2.032215,1.179991
5,0.525713,0.857140,0.981396,-0.438240,0.725576
6,1.747038,-2.069927,0.817761,0.380805,-0.198778


将函数与数组，字典或系列混合使用不是问题，因为所有内容都在内部转换为数组：

In [30]:
key_list = ['one', 'one', 'one', 'two', 'two']

people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -0.631809  0.909270 -0.805212  0.204254 -1.014250
  two  0.008168  0.127904 -1.366128  1.347528  0.290832
5 one  0.525713  0.857140  0.981396 -0.438240  0.725576
6 two  1.747038 -2.069927  0.817761  0.380805 -0.198778

### 按索引级别分组

分层索引数据集的最后一个便利是使用轴索引级别之一进行聚合的能力。让我们看一个例子：

In [31]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                     [1, 3, 5, 1, 3]],
                                     names=['cty', 'tenor'])

hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)

hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.709507 -1.403283 -1.138548  0.407456 -1.483491
1     -1.444128 -0.341516  0.546587 -1.984360 -2.402417
2     -1.522442 -0.216381 -0.317255 -0.354495  0.363017
3     -0.045313 -0.716981  0.858259 -1.174676 -2.079904

要按级别分组，请使用level关键字传递级别编号或名称：

In [32]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 10.2 数据聚合

聚合是指从数组产生标量值的任何数据转换。前面的示例使用了其中的几个，包括均值，计数，最小值和总和。您可能想知道在GroupBy对象上调用mean时发生了什么。许多常见的聚合（例如表10-1中的聚合）已实现了优化。但是，您不仅限于这套方法。

您可以使用自己设计的聚合，还可以调用在分组对象上也定义的任何方法。例如，您可能还记得，分位数是计算Series或DataFrame列的样本分位数。

虽然没有为GroupBy明确实现分位数，但它是一种Series方法，因此可以使用。在内部，GroupBy有效地分割Series，为每个片段调用piece.quantile（0.9），然后将这些结果组装在一起成为结果对象：

In [33]:
df

,key1,key2,data1,data2
0,a,one,-1.299674,0.592971
1,a,two,1.236301,-0.816862
2,b,one,0.247436,0.887067
3,b,two,-0.126520,1.445629
4,a,one,-0.894032,0.495924


In [34]:
grouped = df.groupby('key1')

In [38]:
grouped['data1'].quantile(0.9)

key1
a    0.810234
b    0.210041
Name: data1, dtype: float64

要使用您自己的聚合函数，请将将数组聚合的任何函数传递给aggregate或agg方法：

In [39]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [40]:
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,2.535975,1.409833
b,0.373956,0.558562


您可能会注意到，严格来说，像describe这样的某些方法也可以工作，即使它们不是聚合。

In [41]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.319135  1.362231 -1.299674 -1.096853 -0.894032  0.171135   
b      2.0  0.060458  0.264427 -0.126520 -0.033031  0.060458  0.153947   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     1.236301   3.0  0.090678  0.787449 -0.816862 -0.160469  0.495924   
b     0.247436   2.0  1.166348  0.394963  0.887067  1.026707  1.166348   

                          
           75%       max  
key1                      
a     0.544448  0.592971  
b     1.305988  1.445629

我将在第302页的第10.3节“应用：常规split-apply-combine”中更详细地说明这里发生的情况。

> 自定义聚合函数通常比表10-1中的优化函数慢得多。这是因为在构造中间组数据块时会有一些额外的开销（函数调用，数据重排）。

### 逐列和多函数应用

让我们回到前面示例中的小费数据集。用read_csv加载后，我们添加了一个小费百分比列tip_pct：

In [42]:
tips = pd.read_csv('examples/tips.csv')

In [44]:
# Add tip percentage of total bill
tips['tip_pct'] = tips['tip'] / tips['total_bill']

tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


如您所见，聚合DataFrame的Series或所有列仅是使用具有所需函数的聚合或调用诸如mean或std之类的方法。但是，您可能要根据列使用不同的功能或一次使用多个功能进行汇总。幸运的是，可以做到这一点，我将通过许多示例进行说明。首先，我将按天和吸烟者分类提示：

In [45]:
grouped = tips.groupby(['day', 'smoker'])

请注意，对于表10-1中的描述性统计信息，可以将函数名称作为字符串传递：

In [46]:
grouped_pct = grouped['tip_pct']

In [50]:
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

如果改为传递一个函数列表或函数名称，则返回一个DataFrame，其中的列名称取自这些函数：

In [48]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

在这里，我们向agg传递了一个聚合函数列表，以独立地评估数据组。

您无需接受GroupBy为列指定的名称；值得注意的是，lambda函数的名称为“ <lambda>”，这使它们难以识别（您可以通过查看函数的__name__属性来亲自查看）。因此，如果传递（名称，函数）元组的列表，则每个元组的第一个元素将用作DataFrame列名（您可以将2元组的列表视为有序映射）：

In [51]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

有了DataFrame，您可以有更多选项，因为您可以指定要应用于所有列的功能列表，或者可以指定每列不同的功能。首先，假设我们要为tip_pct和total_bill列计算相同的三个统计信息：

In [59]:
functions = ['count', 'mean', 'max']

result = grouped[['tip_pct', 'total_bill']].agg(functions)

In [60]:
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

如您所见，生成的DataFrame具有分层的列，就像您分别聚合每个列并使用concat将列名作为keys参数将结果粘合在一起一样：

In [61]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

和以前一样，可以传递具有自定义名称的元组列表：

In [62]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]

grouped[['tip_pct', 'total_bill']].agg(ftuples)

tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

现在，假设您要对一个或多个列应用可能不同的功能。为此，将dict传递给agg，其中包含列名到到目前为止列出的任何函数规范的映射：

In [63]:
grouped.agg({'tip': np.max, 'total_bill': 'sum'})

tip  total_bill
day  smoker                   
Fri  No       3.50       73.68
     Yes      4.73      252.20
Sat  No       9.00      884.78
     Yes     10.00      893.62
Sun  No       6.00     1168.88
     Yes      6.50      458.28
Thur No       6.70      770.09
     Yes      5.00      326.24

In [67]:
grouped.agg({'tip': ['min', 'max', 'mean', 'std'],
             'size': 'sum'})

tip                            size
              min    max      mean       std  sum
day  smoker                                      
Fri  No      1.50   3.50  2.812500  0.898494    9
     Yes     1.00   4.73  2.714000  1.077668   31
Sat  No      1.00   9.00  3.102889  1.642088  115
     Yes     1.00  10.00  2.875476  1.630580  104
Sun  No      1.01   6.00  3.167895  1.224785  167
     Yes     1.50   6.50  3.516842  1.261151   49
Thur No      1.25   6.70  2.673778  1.282964  112
     Yes     2.00   5.00  3.030000  1.113491   40

仅当将多个函数应用于至少一列时，DataFrame才会具有层次结构列。

### 返回没有行索引的聚合数据

到目前为止，在所有示例中，聚合数据都返回了一个索引，该索引可能是层次结构的，由唯一的组键组合组成。 由于并非总是如此，因此您可以在大多数情况下通过将as_index = False传递给groupby来禁用此行为：

In [68]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


当然，总是可以通过对结果调用reset_index来获得这种格式的结果。 使用as_index = False方法可以避免不必要的计算。

## 10.3 套用：一般分割套用

最通用的GroupBy方法适用，这是本节其余部分的主题。如图10-2所示，Apply将要操作的对象拆分为多个片段，在每个片段上调用传递的函数，然后尝试将各个片段连接在一起。

![image](images/10-1.bmp)

从以前返回小费数据集，假设您想按组选择前五个tip_pct值。首先，编写一个函数来选择特定列中具有最大值的行：

In [69]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

In [70]:
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


现在，如果我们按吸烟者分组，然后说使用此功能调用Apply，我们将得到以下信息：

In [74]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

这里发生了什么？在DataFrame的每个行组上调用top函数，然后使用pandas.concat将结果粘合在一起，并用组名标记各部分。因此，结果具有一个层次结构索引，其内部级别包含来自原始DataFrame的索引值。

如果您要传递一个采用其他参数或关键字的函数，则可以在函数之后传递这些参数：

In [75]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

> 除了这些基本的使用机制之外，充分利用应用程序可能还需要一些创造力。传递的函数内部发生的事情取决于您；它只需要返回一个pandas对象或一个标量值。 本章的其余部分将主要由示例组成，这些示例向您展示如何使用groupby解决各种问题。

您可能还记得我之前在GroupBy对象上调用了describe：

In [76]:
result = tips.groupby('smoker')['tip_pct'].describe()

In [77]:
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [79]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

在GroupBy内部，当您调用诸如describe之类的方法时，它实际上只是以下各项的快捷方式：

```python
    f = lambda x: x.describe()
    grouped.apply(f)
```

### 禁止组键

在前面的示例中，您看到生成的对象具有由组键以及每个原始对象的索引组成的层次结构索引。您可以通过将group_keys = False传递给groupby来禁用它：

In [82]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


### 分位数和桶分析

您可能会从第8章中回想起，pandas有一些工具，尤其是cut和qcut，可用于将数据切片到带有您选择的分箱或样本分位数的存储桶中。将这些功能与groupby结合使用，可以方便地对数据集执行存储桶或分位数分析。考虑一个简单的随机数据集和使用cut的等长桶分类：

In [83]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)})

In [86]:
quantile = pd.cut(frame.data1, 4)

In [87]:
quantile[:10]

0    (-1.422, 0.0935]
1     (0.0935, 1.609]
2    (-1.422, 0.0935]
3     (0.0935, 1.609]
4    (-1.422, 0.0935]
5     (0.0935, 1.609]
6     (0.0935, 1.609]
7     (0.0935, 1.609]
8    (-1.422, 0.0935]
9    (-1.422, 0.0935]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-2.943, -1.422] < (-1.422, 0.0935] < (0.0935, 1.609] < (1.609, 3.124]]

cut返回的分类对象可以直接传递给groupby。因此我们可以为data2列计算一组统计信息，如下所示：

In [91]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}

In [92]:
grouped = frame.data2.groupby(quantile)

In [93]:
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-2.943, -1.422]",-2.405564,2.462002,96.0,0.043001
"(-1.422, 0.0935]",-2.729503,3.527701,460.0,0.023873
"(0.0935, 1.609]",-3.271560,2.325509,382.0,0.006313
"(1.609, 3.124]",-2.202464,3.191791,62.0,0.120992


这些是等长的水桶。 要基于样本分位数计算大小相等的存储桶，请使用qcut。我将通过labels = False来获得分位数：

In [94]:
grouping = pd.qcut(frame.data1, 10, labels=False)

In [95]:
grouped = frame.data2.groupby(grouping)

In [96]:
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-2.405564,2.462002,100.0,0.023788
1,-2.050014,2.716020,100.0,-0.028333
2,-2.160006,2.847715,100.0,0.057936
3,-2.729503,2.896806,100.0,-0.081330
4,-2.673634,3.527701,100.0,0.117310
5,-2.481006,2.741432,100.0,0.142314
6,-2.619048,2.119694,100.0,0.039026
7,-3.271560,2.325509,100.0,-0.138248
8,-2.175508,2.281567,100.0,0.059965


我们将在第12章中仔细研究熊猫的分类类型。

### 示例：使用特定于组的值填充缺失值

清理丢失的数据时，在某些情况下，您将使用dropna替换数据观察值，但在其他情况下，您可能希望使用固定值或从数据派生的某些值来估算（填充）空值（NA）。fillna是正确使用的工具；例如，在这里我用平均值填充NA值：

In [97]:
s = pd.Series(np.random.randn(6))

In [98]:
s[::2] = np.nan

In [99]:
s

0         NaN
1    0.593852
2         NaN
3    0.746519
4         NaN
5    1.596849
dtype: float64

In [100]:
s.fillna(s.mean())

0    0.979073
1    0.593852
2    0.979073
3    0.746519
4    0.979073
5    1.596849
dtype: float64

假设您需要填充值以按组变化。一种实现方法是对数据进行分组，并与在每个数据块上调用fillna的函数一起使用apply。以下是一些分为东部和西部地区的美国各州的示例数据：

In [101]:
states = ['Ohio', 'New York', 'Vermont', 'Florida', 'Oregon', 'Nevada', 'California', 'Idaho']

In [119]:
group_key = ['East'] * 4 + ['West'] * 4

In [120]:
data = pd.Series(np.random.randn(8), index=states)

In [121]:
data

Ohio          0.958281
New York      1.049614
Vermont       1.566331
Florida       0.862588
Oregon        1.159276
Nevada        1.171808
California   -1.548883
Idaho        -1.171751
dtype: float64

请注意，语法['East'] * 4会产生一个包含['East']中元素的四个副本的列表。 将列表加在一起将它们串联在一起。

让我们在数据中设置一些缺少的值：

In [122]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan

In [123]:
data

Ohio          0.958281
New York      1.049614
Vermont            NaN
Florida       0.862588
Oregon        1.159276
Nevada             NaN
California   -1.548883
Idaho              NaN
dtype: float64

In [124]:
data.groupby(group_key).mean()

East    0.956828
West   -0.194803
dtype: float64

我们可以使用以下组均值来填充NA值：

In [125]:
fill_mean = lambda g: g.fillna(g.mean())

In [126]:
data.groupby(group_key).apply(fill_mean)

Ohio          0.958281
New York      1.049614
Vermont       0.956828
Florida       0.862588
Oregon        1.159276
Nevada       -0.194803
California   -1.548883
Idaho        -0.194803
dtype: float64

在另一种情况下，您的代码中可能会有预定义的填充值，该填充值随组而变化。 由于组在内部设置了名称属性，因此我们可以使用它：

In [127]:
fill_values = {'East': 0.5, 'West': -1}

In [128]:
fill_func = lambda g: g.fillna(fill_values[g.name])

In [129]:
data.groupby(group_key).apply(fill_func)

Ohio          0.958281
New York      1.049614
Vermont       0.500000
Florida       0.862588
Oregon        1.159276
Nevada       -1.000000
California   -1.548883
Idaho        -1.000000
dtype: float64

### 示例：随机采样和置换

假设您要从大型数据集中绘制一个随机样本（有或没有替换），以进行蒙特卡洛模拟或其他应用。有多种方法可以执行“抽奖”；这里我们使用Series的样本方法。

为了演示，这是一种构造一副英式扑克牌的方法：

In [135]:
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'Q', 'K']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)

因此，现在我们有了一个长度为52的系列，其索引包含纸牌名称，而值是二十一点和其他游戏中使用的纸牌名称（为简单起见，我将ace'A'设为1）：

In [136]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
QH     10
KH     10
dtype: int64

现在，根据我之前所说的，从牌组中抽出五张牌的手可以写成：

In [137]:
def draw(deck, n=5):
    return deck.sample(n)

In [138]:
draw(deck)

JH    10
4H     4
3D     3
9H     9
6H     6
dtype: int64

假设您想从每套西装中随机抽取两张卡片。 因为西装是每个卡名的最后一个字符，所以我们可以基于此分组并使用apply：

In [139]:
get_suit = lambda card: card[-1] # last letter is suit

In [141]:
deck.groupby(get_suit).apply(draw, n=5)

C  4C     4
   7C     7
   3C     3
   AC     1
   5C     5
D  8D     8
   JD    10
   9D     9
   6D     6
   5D     5
H  8H     8
   JH    10
   KH    10
   7H     7
   2H     2
S  JS    10
   3S     3
   9S     9
   7S     7
   QS    10
dtype: int64

或者，我们可以这样写：

In [142]:
deck.groupby(get_suit, group_keys=False).apply(draw, 2)

KC     10
6C      6
JD     10
6D      6
8H      8
7H      7
2S      2
10S    10
dtype: int64

### 示例：组加权平均值和相关性

在groupby的split-apply-combine范式下，可以在DataFrame或两个Series中的列之间进行操作，例如组加权平均值。 例如，以包含组键，值和一些权重的数据集为例：

In [144]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a', 'b', 'b', 'b', 'b'],
                   'data': np.random.randn(8),
                   'weights': np.random.rand(8)})

In [145]:
df

,category,data,weights
0,a,-1.007511,0.502748
1,a,0.420193,0.683388
2,a,-1.094717,0.448615
3,a,-1.953998,0.627799
4,b,-0.467096,0.789669
5,b,1.085136,0.005286
6,b,0.601202,0.725480
7,b,1.406553,0.282152


这样，按类别划分的组加权平均值将为：

In [146]:
grouped = df.groupby('category')

In [147]:
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])

In [148]:
grouped.apply(get_wavg)

category
a   -0.856199
b    0.260685
dtype: float64

作为另一个示例，请考虑最初从Yahoo!获得的财务数据集。 包含几只股票的日末价格和S＆P 500指数（SPX符号）的财务信息：

In [153]:
close_px = pd.read_csv('examples/stock_px_2.csv', parse_dates=True, index_col=0)

In [154]:
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [155]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


一项有趣的任务可能是计算一个数据框架，该数据框架包含每日收益与SPX的年度相关性（根据变化百分比计算）。作为执行此操作的一种方法，我们首先创建一个函数来计算每个列与“ SPX”列的成对相关性：

In [156]:
spx_corr = lambda x: x.corrwith(x['SPX'])

接下来，我们使用pct_change计算close_px的变化百分比：

In [157]:
rets = close_px.pct_change().dropna()

最后，我们按年份对这些百分比变化进行分组，可以使用单行函数从每个行标签中提取这些百分比变化，该函数返回每个日期时间标签的year属性：

In [159]:
get_year = lambda x: x.year

by_year = rets.groupby(get_year)

by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


您还可以计算列间关联。这里我们计算苹果与微软之间的年度相关性：

In [160]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### 示例：群智线性回归

在与上一个示例相同的主题中，您可以使用groupby来执行更复杂的按组统计分析，只要该函数返回pandas对象或标量值即可。 例如，我可以定义以下回归函数（使用statsmodels econometrics库），该函数对每个数据块执行普通最小二乘（OLS）回归：

In [163]:
import statsmodels.api as sm

def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

现在，要对SPX返回值运行AAPL的年度线性回归，请执行：

In [164]:
by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514
